# Fe55クラスターの平衡化@1500K
FAIRのNNPを用いたMD simulationの練習を兼ねた計算。　　

参考：https://docs.matlantis.com/atomistic-simulation-tutorial/ja/6_2_md-nvt.html

In [ ]:
from fairchem.core import OCPCalculator

import ase
from ase.optimize import LBFGS
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution, Stationary
from ase.md.langevin import Langevin
from ase.md import MDLogger
from ase import units
from time import perf_counter
import os

#　クラスター初期構造の読み込み
atoms =  ase.io.read("../structure_build/Fe55.xyz")

# 学習済みモデル(NNP)の読み込み
calculator = OCPCalculator(
    model_name="EquiformerV2-31M-S2EF-OC20-All+MD", # 申請が通ったらOMat24のモデルに変更
    local_cache="pretrained_models",
    cpu=True,
)

atoms.calc = calculator

# 構造緩和
opt = LBFGS(atoms)
opt.run(fmax=0.05)

# MD計算の条件
time_step    = 1.0    # fsec
temperature  = 1500   # Kelvin
num_md_steps = 100     # Total MD step, for testing.
# num_md_steps = 1e6    # Total MD step, for actual run.
num_interval = 1      # Output printing interval
friction_coeff = 0.005

# 出力設定
output_dir = "output"
# ディレクトリが存在しない場合のみ作成
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
output_filename = f"./output/{atoms.symbols}_{temperature}K"

print("output_filename = ",output_filename)
log_filename = output_filename + ".log"
print("log_filename = ",log_filename)
traj_filename = output_filename + ".traj"
print("traj_filename = ",traj_filename)

# 初速の設定
MaxwellBoltzmannDistribution(atoms, temperature_K=temperature,force_temp=True)
Stationary(atoms)  # 運動量の総和をゼロベクトルにする

# run MD
dyn = Langevin(atoms, time_step*units.fs, friction=friction_coeff, temperature_K=temperature, loginterval=num_interval, trajectory=traj_filename)

# Print statements
def print_dyn():
    imd = dyn.get_number_of_steps()
    etot  = atoms.get_total_energy()
    temp_K = atoms.get_temperature()
    elapsed_time = perf_counter() - start_time
    print(f"  {imd: >3}   {etot:.3f}    {temp_K:.2f}   {elapsed_time:.3f}")


dyn.attach(print_dyn, interval=num_interval)
dyn.attach(MDLogger(dyn, atoms, log_filename, header=True, stress=False, peratom=True, mode="w"), interval=num_interval)

# Now run the dynamics
start_time = perf_counter()
print(f"    imd     Etot(eV)    T(K)    elapsed_time(sec)")
dyn.run(num_md_steps)


INFO:root:Checking local cache: pretrained_models for model EquiformerV2-31M-S2EF-OC20-All+MD
/home/oxygen/miniforge3/envs/fair-chem-cpu/lib/python3.12/site-packages/fairchem/core/common/relaxation/ase_utils.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded fil

       Step     Time          Energy          fmax
LBFGS:    0 20:23:36      106.226669       10.241766
LBFGS:    1 20:23:37       52.717407        6.003483
LBFGS:    2 20:23:38        5.957617        2.339198
LBFGS:    3 20:23:38       -0.359710        0.943141
LBFGS:    4 20:23:39       -0.744447        0.264595
LBFGS:    5 20:23:40       -0.778529        0.163982
LBFGS:    6 20:23:41       -0.801449        0.029292
output_filename =  ./output/Fe55_1500K
log_filename =  ./output/Fe55_1500K.log
traj_filename =  ./output/Fe55_1500K.traj
    imd     Etot(eV)    T(K)    elapsed_time(sec)
    0   9.862    1500.00   0.002
    1   9.948    1512.02   0.753
    2   9.922    1507.13   1.551
    3   9.893    1499.33   2.326
    4   9.834    1484.55   3.103
    5   9.749    1462.58   3.904
    6   9.680    1437.33   4.718
    7   9.525    1391.12   5.545
    8   9.511    1356.64   6.327
    9   9.468    1312.77   7.089
   10   9.402    1263.51   7.898
   11   9.374    1217.71   8.661
   12   9.3

True

In [13]:
from ase.io import Trajectory
from ase.io.trajectory import TrajectoryReader
import sys

path = "../visualization"
sys.path.append(os.path.abspath(path))
from ase_nglview import view_ngl

traj = TrajectoryReader(traj_filename)
view_ngl(traj)

NGLWidget(max_frame=100)